# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,4.00,82,100,2.88,GS,1736445377
1,1,vorkuta,67.5000,64.0000,-14.50,88,100,6.58,RU,1736445378
2,2,sola,-13.8833,167.5500,26.49,81,98,2.10,VU,1736445379
3,3,elverum,60.8819,11.5623,-8.74,96,99,1.55,NO,1736445380
4,4,stanley,54.8680,-1.6985,-1.57,81,0,4.63,GB,1736445382


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]


# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
116,116,east london,-33.0153,27.9116,25.52,73,0,3.09,ZA,1736445524
235,235,zinjibar,13.1287,45.3807,24.32,72,0,3.78,YE,1736450070
296,296,port alfred,-33.5906,26.8910,22.32,94,0,0.45,ZA,1736450456
302,302,akjoujt,19.6969,-14.4164,24.61,11,0,3.92,MR,1736450463
315,315,ormara,25.2088,64.6357,21.40,49,0,0.12,PK,1736450479
429,429,salalah,17.0151,54.0924,22.05,64,0,1.54,OM,1736450617
457,457,isangel,-19.5500,169.2667,23.88,72,0,3.95,VU,1736450653


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
116,east london,ZA,-33.0153,27.9116,73,
235,zinjibar,YE,13.1287,45.3807,72,
296,port alfred,ZA,-33.5906,26.8910,94,
302,akjoujt,MR,19.6969,-14.4164,11,
315,ormara,PK,25.2088,64.6357,49,
429,salalah,OM,17.0151,54.0924,64,
457,isangel,VU,-19.5500,169.2667,72,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 20


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    filter= f"circle:{longitude},{latitude},{radius}"
    bias= f"proximity:{longitude},{latitude}"
    params = {
    "filter":filter,
    "limit":limit,
    "bias":bias,
    "apiKey":geoapify_key    
    }
    params["categories"] = f"accommodation.hotel"
   
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
east london - nearest hotel: No hotel found
zinjibar - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
akjoujt - nearest hotel: Hotel Fatima فندق فاطمة
ormara - nearest hotel: No hotel found
salalah - nearest hotel: Muscat International Hotel
isangel - nearest hotel: Tanna Lodge


,City,Country,Lat,Lng,Humidity,Hotel Name
116,east london,ZA,-33.0153,27.9116,73,No hotel found
235,zinjibar,YE,13.1287,45.3807,72,No hotel found
296,port alfred,ZA,-33.5906,26.8910,94,No hotel found
302,akjoujt,MR,19.6969,-14.4164,11,Hotel Fatima فندق فاطمة
315,ormara,PK,25.2088,64.6357,49,No hotel found
429,salalah,OM,17.0151,54.0924,64,Muscat International Hotel
457,isangel,VU,-19.5500,169.2667,72,Tanna Lodge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name","Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)